 # 📌 Task 1: End-to-End ML Pipeline with Scikit-learn Pipeline API

## ✅ Objective
Build a reusable and production-ready machine learning pipeline to predict customer churn using the Telco Churn dataset.

## 📚 Dataset
**Telco Customer Churn Dataset**  
This dataset contains customer information including demographics, account details, services subscribed, and churn status.

## 🛠️ Tools & Libraries
- Python
- Pandas & NumPy
- Scikit-learn (Pipeline, ColumnTransformer, GridSearchCV)
- Joblib (for model export)

## 🧠 Skills Gained
- Building production-ready ML pipelines  
- Preprocessing using `Pipeline` and `ColumnTransformer`  
- Model training using Logistic Regression and Random Forest  
- Hyperparameter tuning using `GridSearchCV`  
- Model export & reusability using `joblib`

## 📌 Evaluation Metrics
- **Accuracy**
- **F1 Score**
- **Classification Report**

## 🧪 Output
- Best trained model saved as **churn_pipeline_model.pkl**


### Import Libraries

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import accuracy_score, f1_score, classification_report
import joblib


### Load Dataset

In [2]:
df = pd.read_csv("Telco-Customer-Churn.csv")
df.head()


,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


### Data Cleaning

In [3]:
# Replace spaces with NaN in TotalCharges column
df["TotalCharges"] = df["TotalCharges"].replace(" ", np.nan)

# Convert to numeric
df["TotalCharges"] = pd.to_numeric(df["TotalCharges"])

# Fill missing values
df["TotalCharges"].fillna(df["TotalCharges"].median(), inplace=True)

# Convert target to binary
df["Churn"] = df["Churn"].replace({"Yes": 1, "No": 0})

df.head()


C:\Users\Asfa\AppData\Local\Temp\ipykernel_16568\2185977910.py:8: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["TotalCharges"].fillna(df["TotalCharges"].median(), inplace=True)
C:\Users\Asfa\AppData\Local\Temp\ipykernel_16568\2185977910.py:11: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df["Ch

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,0
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.50,0
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,1
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,0
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,1


### Train-Test Split

In [4]:
X = df.drop("Churn", axis=1)
y = df["Churn"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)


### Build Preprocessing Pipeline

In [5]:
# Identify numerical and categorical columns
num_cols = X.select_dtypes(include=["int64", "float64"]).columns
cat_cols = X.select_dtypes(include=["object"]).columns

# Preprocessing pipeline
num_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

cat_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("encoder", OneHotEncoder(handle_unknown="ignore"))
])

preprocessor = ColumnTransformer([
    ("num", num_pipeline, num_cols),
    ("cat", cat_pipeline, cat_cols)
])


### Model Pipelines

In [7]:
log_pipeline = Pipeline([
    ("preprocessor", preprocessor),
    ("classifier", LogisticRegression(max_iter=500))
])

rf_pipeline = Pipeline([
    ("preprocessor", preprocessor),
    ("classifier", RandomForestClassifier())
])



### Hyperparameter Tuning with GridSearchCV

In [8]:
log_params = {
    "classifier__C": [0.01, 0.1, 1, 10],
    "classifier__penalty": ["l1", "l2"],
    "classifier__solver": ["liblinear"]
}

log_grid = GridSearchCV(
    log_pipeline,
    log_params,
    cv=5,
    scoring="f1",
    n_jobs=-1
)

log_grid.fit(X_train, y_train)


,estimator,Pipeline(step...x_iter=500))])
,param_grid,"{'classifier__C': [0.01, 0.1, ...], 'classifier__penalty': ['l1', 'l2'], 'classifier__solver': ['liblinear']}"
,scoring,'f1'
,n_jobs,-1
,refit,True
,cv,5
,verbose,0
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,transformers,"[('num', ...), ('cat', ...)]"


In [9]:
rf_params = {
    "classifier__n_estimators": [50, 100, 200],
    "classifier__max_depth": [5, 10, 20],
    "classifier__min_samples_split": [2, 5, 10]
}

rf_grid = GridSearchCV(
    rf_pipeline,
    rf_params,
    cv=5,
    scoring="f1",
    n_jobs=-1
)

rf_grid.fit(X_train, y_train)


,estimator,Pipeline(step...lassifier())])
,param_grid,"{'classifier__max_depth': [5, 10, ...], 'classifier__min_samples_split': [2, 5, ...], 'classifier__n_estimators': [50, 100, ...]}"
,scoring,'f1'
,n_jobs,-1
,refit,True
,cv,5
,verbose,0
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,transformers,"[('num', ...), ('cat', ...)]"


### Evaluation

In [10]:
def evaluate(model, X_test, y_test):
    y_pred = model.predict(X_test)
    print("Accuracy:", accuracy_score(y_test, y_pred))
    print("F1 Score:", f1_score(y_test, y_pred))
    print("\nClassification Report:\n", classification_report(y_test, y_pred))

print("=== Logistic Regression Results ===")
evaluate(log_grid.best_estimator_, X_test, y_test)

print("\n=== Random Forest Results ===")
evaluate(rf_grid.best_estimator_, X_test, y_test)


=== Logistic Regression Results ===
Accuracy: 0.8019872249822569
F1 Score: 0.5974025974025974

Classification Report:
               precision    recall  f1-score   support

           0       0.85      0.89      0.87      1035
           1       0.65      0.55      0.60       374

    accuracy                           0.80      1409
   macro avg       0.75      0.72      0.73      1409
weighted avg       0.79      0.80      0.80      1409


=== Random Forest Results ===
Accuracy: 0.794889992902768
F1 Score: 0.48484848484848486

Classification Report:
               precision    recall  f1-score   support

           0       0.81      0.95      0.87      1035
           1       0.73      0.36      0.48       374

    accuracy                           0.79      1409
   macro avg       0.77      0.66      0.68      1409
weighted avg       0.78      0.79      0.77      1409



### Save the Best Pipeline

In [11]:
best_model = rf_grid.best_estimator_  # or log_grid.best_estimator_

joblib.dump(best_model, "churn_pipeline_model.pkl")
print("Model saved successfully!")


Model saved successfully!


### Load & Test Saved Model

In [12]:
loaded_model = joblib.load("churn_pipeline_model.pkl")

# Test on sample data
sample = X_test.iloc[0:1]
print("Prediction:", loaded_model.predict(sample))
print("Actual:", y_test.iloc[0])


Prediction: [0]
Actual: 0


### Final Summary / Insights

### Summary
- Data preprocessing was done using Scikit-learn Pipeline.
- Logistic Regression and Random Forest models were trained.
- Hyperparameter tuning was done using GridSearchCV.
- Best model was exported using joblib for production use.
